In [1]:
import pandas as pd

In [2]:
raw_votes = pd.read_csv('votematrix-2015.dat', delimiter='\t')

In [3]:
raw_votes.ix[:5,:10]

,rowid,date,voteno,Bill,mpid40706,mpid40707,mpid40708,mpid40709,mpid40710,mpid40711
0,32966,2017-03-29,193,Pension Schemes Bill [Lords] - Systems and pro...,2,-9,2,4,4,2
1,32965,2017-03-29,192,Pension Schemes Bill [Lords] - Member trustees,2,-9,2,4,4,2
2,32964,2017-03-29,191,Pension Schemes Bill [Lords] - Funder of the l...,2,-9,2,4,4,2
3,32958,2017-03-27,190,Bus Services Bill [Lords] - Bus companies: lim...,2,-9,2,4,4,2
4,32957,2017-03-27,189,Bus Services Bill [Lords] - Report on the prov...,2,-9,2,4,4,2
5,32956,2017-03-27,188,Bus Services Bill [Lords] - National strategy,2,-9,2,4,4,2


In this dataset, each vote is associated with a categorical value:
* Minority(2)
* Majority(4)
* Tellaye(1)
* Tellno(5)
* Both(3)
* Absent(-9)

While these codes will probably not be helpful for my eventual analysis, I will keep them in their raw format for now. As the purpose of this data munging process is to create an exportable csv file of this dataset, and it is important to keep the intricities of the UK parliament intact for the moment.

In [5]:
votes = raw_votes.drop(['voteno','date','Bill'], axis=1)
votes = votes.set_index('rowid')
votes = votes.transpose()
votes = votes.reset_index()
votes = votes.rename(columns={'index':'mpid'})
votes = votes.drop(votes.index[654])
votes.columns.name = ''
votes.ix[:5,:10]

,mpid,32966,32965,32964,32958,32957,32956,32955,32949,32944
0,mpid40706,2,2,2,2,2,2,-9,-9,4
1,mpid40707,-9,-9,-9,-9,-9,-9,-9,-9,4
2,mpid40708,2,2,2,2,2,2,-9,-9,4
3,mpid40709,4,4,4,4,4,4,-9,-9,2
4,mpid40710,4,4,4,4,4,4,-9,-9,2
5,mpid40711,2,2,2,2,2,2,2,-9,4


In [6]:
import re
def mpid_normalizer (raw_value):
    numbers_only = re.sub("[^0-9]","", raw_value)
    try:
        mp_value = int(numbers_only)
    except:
        numbers_only = 0
        mp_value = int(numbers_only)
    return(mp_value)

In [7]:
votes['mpid'] = votes['mpid'].apply(mpid_normalizer)
votes.tail()

,mpid,32966,32965,32964,32958,32957,32956,32955,32949,32944,...,32110,32109,32108,32106,32105,32104,32103,32107,32101,32100
649,41366,4,4,4,4,4,4,-9,2,2,...,-9,-9,-9,-9,-9,-9,-9,-9,-9,-9
650,41367,2,2,2,2,2,2,-9,-9,-9,...,-9,-9,-9,-9,-9,-9,-9,-9,-9,-9
651,41368,4,4,4,-9,-9,-9,-9,2,2,...,-9,-9,-9,-9,-9,-9,-9,-9,-9,-9
652,41369,4,4,4,4,4,4,-9,2,2,...,-9,-9,-9,-9,-9,-9,-9,-9,-9,-9
653,41370,2,2,2,2,2,2,-9,-9,4,...,-9,-9,-9,-9,-9,-9,-9,-9,-9,-9


In [8]:
votes = votes.astype(int)

In [10]:
raw_mpid = pd.read_json('mps.js')

In [11]:
raw_mpid.head()

,constituency,member_id,name,office,party,person_id
0,Houghton and Sunderland South,40706,Bridget Phillipson,"[{'from_date': '2010-10-11', 'to_date': '9999-...",Labour,24709
1,Sunderland Central,40707,Julie Elliott,"[{'from_date': '2015-10-26', 'to_date': '9999-...",Labour,24710
2,Washington and Sunderland West,40708,Sharon Hodgson,"[{'from_date': '2016-10-09', 'to_date': '9999-...",Labour,11592
3,North Swindon,40709,Justin Tomlinson,NaN,Conservative,24870
4,Putney,40710,Justine Greening,"[{'from_date': '2016-07-14', 'to_date': '9999-...",Conservative,11771


In [20]:
mpid = raw_mpid.drop(['office', 'person_id'], axis=1)
mpid = mpid.rename(columns={'member_id':'mpid'})
mpid['mpid'] = mpid['mpid'].astype(int)
cols = ['mpid', 'name', 'constituency', 'party']
mpid = mpid[cols]

In [21]:
mpid.head()

,mpid,name,constituency,party
0,40706,Bridget Phillipson,Houghton and Sunderland South,Labour
1,40707,Julie Elliott,Sunderland Central,Labour
2,40708,Sharon Hodgson,Washington and Sunderland West,Labour
3,40709,Justin Tomlinson,North Swindon,Conservative
4,40710,Justine Greening,Putney,Conservative


In [22]:
voting_record = pd.merge(mpid, votes, how='left', on='mpid')

In [23]:
voting_record.head()

,mpid,name,constituency,party,32966,32965,32964,32958,32957,32956,...,32110,32109,32108,32106,32105,32104,32103,32107,32101,32100
0,40706,Bridget Phillipson,Houghton and Sunderland South,Labour,2.0,2.0,2.0,2.0,2.0,2.0,...,-9.0,2.0,2.0,-9.0,2.0,-9.0,4.0,-9.0,2.0,2.0
1,40707,Julie Elliott,Sunderland Central,Labour,-9.0,-9.0,-9.0,-9.0,-9.0,-9.0,...,-9.0,2.0,2.0,2.0,2.0,-9.0,-9.0,-9.0,-9.0,2.0
2,40708,Sharon Hodgson,Washington and Sunderland West,Labour,2.0,2.0,2.0,2.0,2.0,2.0,...,-9.0,2.0,2.0,2.0,2.0,-9.0,4.0,-9.0,2.0,2.0
3,40709,Justin Tomlinson,North Swindon,Conservative,4.0,4.0,4.0,4.0,4.0,4.0,...,4.0,4.0,4.0,4.0,2.0,4.0,2.0,4.0,4.0,4.0
4,40710,Justine Greening,Putney,Conservative,4.0,4.0,4.0,4.0,4.0,4.0,...,4.0,4.0,4.0,4.0,2.0,4.0,-9.0,-9.0,-9.0,4.0


In [24]:
voting_record.tail()

,mpid,name,constituency,party,32966,32965,32964,32958,32957,32956,...,32110,32109,32108,32106,32105,32104,32103,32107,32101,32100
644,41366,Robert Courts,Witney,Conservative,4.0,4.0,4.0,4.0,4.0,4.0,...,-9.0,-9.0,-9.0,-9.0,-9.0,-9.0,-9.0,-9.0,-9.0,-9.0
645,41367,Sarah Olney,Richmond Park,Liberal Democrat,2.0,2.0,2.0,2.0,2.0,2.0,...,-9.0,-9.0,-9.0,-9.0,-9.0,-9.0,-9.0,-9.0,-9.0,-9.0
646,41368,Dr Caroline Johnson,Sleaford and North Hykeham,Conservative,4.0,4.0,4.0,-9.0,-9.0,-9.0,...,-9.0,-9.0,-9.0,-9.0,-9.0,-9.0,-9.0,-9.0,-9.0,-9.0
647,41369,Trudy Harrison,Copeland,Conservative,4.0,4.0,4.0,4.0,4.0,4.0,...,-9.0,-9.0,-9.0,-9.0,-9.0,-9.0,-9.0,-9.0,-9.0,-9.0
648,41370,Gareth Snell,Stoke-on-Trent Central,Labour,2.0,2.0,2.0,2.0,2.0,2.0,...,-9.0,-9.0,-9.0,-9.0,-9.0,-9.0,-9.0,-9.0,-9.0,-9.0


In [25]:
voting_record.to_csv('2015parliament.csv', index=False)